<a href="https://colab.research.google.com/github/acpuche/web_scraping_genai/blob/origin%2Fparsing/web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup

In [6]:
#Requirements
#!pip install asyncio==3.4.3
#!pip install Crawl4AI==0.6.3
#!pip install Unidecode==1.4.0
#!pip install json-repair -q

# run playwright install

!pip install -r /content/requirements.txt -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.0/93.0 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the s

In [10]:
!playwright install

171.6 MiB [] 0% 11.4s171.6 MiB [] 0% 50.1s171.6 MiB [] 0% 25.5s171.6 MiB [] 0% 16.1s171.6 MiB [] 0% 9.1s171.6 MiB [] 1% 5.9s171.6 MiB [] 1% 5.3s171.6 MiB [] 2% 4.3s171.6 MiB [] 3% 3.7s171.6 MiB [] 4% 3.2s171.6 MiB [] 5% 3.0s171.6 MiB [] 5% 3.2s171.6 MiB [] 6% 3.1s171.6 MiB [] 6% 3.2s171.6 MiB [] 6% 3.3s171.6 MiB [] 7% 3.2s171.6 MiB [] 8% 3.3s171.6 MiB [] 8% 3.2s171.6 MiB [] 9% 3.2s171.6 MiB [] 9% 3.3s171.6 MiB [] 10% 3.2s171.6 MiB [] 10% 3.1s171.6 MiB [] 11% 3.2s171.6 MiB [] 11% 3.1s171.6 MiB [] 12% 3.1s171.6 MiB [] 13% 3.0s171.6 MiB [] 14% 3.0s171.6 MiB [] 14% 2.9s171.6 MiB [] 15% 2.8s171.6 MiB [] 16% 2.8s171.6 MiB [] 17% 2.7s171.6 MiB [] 18% 2.6s171.6 MiB [] 19% 2.6s171.6 MiB [] 19% 2.5s171.6 MiB [] 19% 2.6s171.6 MiB [] 20% 2.5s171.6 MiB [] 21% 2.5s171.6 MiB [] 22% 2.4s171.6 MiB [] 22% 2.5s171.6 MiB [] 22% 2.6s171.6 MiB [] 23% 2.4s171.6 MiB [] 24% 2.4s171.6 MiB [] 25% 2.3s171.6 MiB [] 26% 2.3s171.6 MiB [] 27% 2.2s171.6 MiB [] 28% 2.2s171.6 MiB [] 29% 2.1s171.6 MiB [] 30% 2.0s171.6 Mi

## Paso 1
Muestreo de urls internas de un sitio de e-commerce, como salida se genera un json con el nombre del comercio y la hora de generación.

La busqueda está limitada por el parámetro maxlen=50 para reducir costos al momento de usar los modelos de lenguaje.




In [ ]:
"https://www.amazon.com/"
"https://www.mercadolibre.com.co/"
"https://www.shein.com.co/"
"https://www.farmatodo.com.co/"

In [54]:
e_commerce_url="https://www.mercadolibre.com.co/"

In [55]:
import json
import asyncio
import datetime
from unidecode import unidecode
from collections import deque
from typing import Dict, List, Set
from crawl4ai.models import CrawlResult
from crawl4ai import AsyncWebCrawler
from urllib.parse import urljoin, urlparse, urlunparse

class SimpleWebsiteScraper:
    def __init__(self, crawler: AsyncWebCrawler):
        self.crawler = crawler
        self.base_url = None

    def is_valid_internal_link(self, link: str) -> bool:
        if not link or link.startswith('#'):
            return False

        parsed_base = urlparse(self.base_url)
        parsed_link = urlparse(link)

        return (parsed_base.netloc == parsed_link.netloc and
                parsed_link.path not in ['', '/'] and
                parsed_link.path.startswith(parsed_base.path))

    def normalize_url(self, url: str) -> str:
        parsed = urlparse(url)
        # Remove any fragments
        parsed = parsed._replace(fragment='')
        # Ensure the path doesn't end with a slash unless it's the root
        if parsed.path.endswith('/') and len(parsed.path) > 1:
            parsed = parsed._replace(path=parsed.path.rstrip('/'))
        return urlunparse(parsed)

    def join_url(self, base: str, url: str) -> str:
        joined = urljoin(base, url)
        parsed_base = urlparse(self.base_url)
        parsed_joined = urlparse(joined)

        # Ensure the joined URL starts with the base path
        if not parsed_joined.path.startswith(parsed_base.path):
            # If it doesn't, prepend the base path
            new_path = parsed_base.path.rstrip('/') + '/' + parsed_joined.path.lstrip('/')
            parsed_joined = parsed_joined._replace(path=new_path)

        return urlunparse(parsed_joined)

    async def scrape(self, start_url: str, max_depth: int) -> Dict[str, CrawlResult]:
        self.base_url = self.normalize_url(start_url)
        results: Dict[str, CrawlResult] = {}
        queue: deque = deque([(self.base_url, 0)],maxlen=50)
        visited: Set[str] = set()

        while queue:

          current_url, current_depth = queue.popleft()
          result = await self.crawler.arun(current_url)

          if current_url in visited or current_depth > max_depth:
            continue

          visited.add(current_url)
          result = await self.crawler.arun(current_url)

          internal_links_test = result.links.get('internal', [])
          for link_test in internal_links_test:
            link_test['text']=unidecode(link_test['text'])

          if result.success:
            results[current_url] = result

            if current_depth < max_depth:
              internal_links = result.links.get('internal', [])
              for link in internal_links:
                full_url = self.join_url(current_url, link['href'])
                normalized_url = self.normalize_url(full_url)
                if self.is_valid_internal_link(normalized_url) and normalized_url not in visited:
                  queue.append((normalized_url, current_depth + 1))

        return results

async def main(start_url: str, depth: int):
    all_results=[]

    async with AsyncWebCrawler() as crawler:
        scraper = SimpleWebsiteScraper(crawler)
        results = await scraper.scrape(start_url, depth)

    # Get the current date and time
    now = datetime.datetime.now()

    # Format the date and time as a string (e.g., "2025-06-23_14-30-00")
    formatted_datetime = now.strftime("%Y-%m-%d_%H-%M-%S")

    # Para varias páginas, hay inconveniente con el append del json y posterior uso.
    print(f"Crawled {len(results)} pages:")
    for url, result in results.items():
        print(f"- {url}: {len(result.links.get('internal', []))} internal links, {len(result.links.get('external', []))} external links")
        print(f"Session: {result.session_id}", )

        """
        with open(f"{start_url.split('.')[1]}_{formatted_datetime}.json", "w") as f:
          all_results.append(result.links)
          json.dump(all_results, f, indent=4)
        """
    json_name = f"{start_url.split('.')[1]}_{formatted_datetime}.json"

    with open(json_name, "w") as f:
      json.dump(result.links, f, indent=4)

    return results, json_name

if __name__ == "__main__":
    start_url = e_commerce_url
    depth = 1
    results, json_name=await(main(start_url, depth))

[INIT].... → Crawl4AI 0.6.3 

[FETCH]... ↓ https://www.mercadolibre.com.co/                                                                     |
✓ | ⏱: 4.09s 

[SCRAPE].. ◆ https://www.mercadolibre.com.co/                                                                     |
✓ | ⏱: 0.49s 

[COMPLETE] ● https://www.mercadolibre.com.co/                                                                     |
✓ | ⏱: 4.61s 

[FETCH]... ↓ https://www.mercadolibre.com.co/                                                                     |
✓ | ⏱: 2.60s 

...

[COMPLETE] ● https://www.mercadolibre.com.co/ofertas/navidad                                                      |
✓ | ⏱: 2.95s 

[FETCH]... ↓ https://www.mercadolibre.com.co/ofertas/navidad                                                      |
✓ | ⏱: 2.68s 

[SCRAPE].. ◆ https://www.mercadolibre.com.co/ofertas/navidad                                                      |
✓ | ⏱: 0.54s 

[COMPLETE] ● https://www.mercadolibre.com.co/ofertas/navidad                                                      |
✓ | ⏱: 3.23s 

Crawled 51 pages:
- https://www.mercadolibre.com.co/: 264 internal links, 15 external links
Session: None
- https://www.mercadolibre.com.co/c/salud-y-equipamiento-medico: 362 internal links, 14 external links
Session: None
- https://www.mercadolibre.com.co/c/servicios: 294 internal links, 15 external links
Session: None
- https://www.mercadolibre.com.co/c/otras-categorias: 227 internal links, 14 external links
Session: None
- https://www.mercadolibre.com.co/pagar-online-con-mercadopago: 20 internal links, 10 external links
Session: None
- https://www.mercadolibre.com.co/seguridad: 15 internal links, 5 external links
Session: None
- https://www.mercadolibre.com.co/glossary/A/1: 365 internal links, 14 external links
Session: None
- https://www.mercadolibre.com.co/glossary/B/1: 371 internal links, 14 external links
Session: None
- https://www.mercadolibre.com.co/glossary/C/1: 371 internal links, 14 external links
Session: None
- https://www.mercadolibre.com.co/glossary/D/1: 369 internal l

## Paso 2
#### Uso de LLM para filtrado de contenido, se parte de una lista de urls del sitio de e-commerce para filtrar el contenido de interés: productos.

In [12]:
!pip install -q -U google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 5.8 MB/s eta 0:00:00


#### Costo por millón de tokens de entrada y salida para el modelo Gemini 2.0 flash

In [47]:
gemini_2_0_flash_input_cost = 0.10
gemini_2_0_flash_output_cost = 0.40

#Utils

#### Parseo de la respuesta del modelo de formato json


In [136]:
def parse_to_dictionary(text_to_parse):
  """
  método que parsea un string a un diccionario cono diferentes estrategias
  """
  parsing_strategies = [
      lambda x: eval(x), # Generic eval for python dictionaries
      lambda x: json.loads(re.sub(r'\\n', '\n', re.sub(r'\\"', '"', re.sub(r'\\\\', '\\',re.search(r'```json\\n(.*?)```', x, re.DOTALL).group(1))))),
      lambda x: json.loads(json.loads(x.strip('"')).replace("```json\\n", "").replace("\\n```", "")),
      lambda x: json.loads(x.replace("\n", "").strip(".\n")), # Clean and parse JSON directly
      lambda x: json.loads(re.search(r'```json\n(.*?)\n```', x, re.DOTALL).group(1)),
      lambda x: eval(re.search(r"```json\n(.*?)\n```", x, re.DOTALL).group(1)),
      lambda x: json.loads(re.search(r"\{.*\}", x, re.DOTALL.group(0))), # Extract shortest JSON-like object with eval
      lambda x: eval(re.search(r"\{.*\}", x, re.DOTALL).group(0)), #Extract JSON-like content usinng eval.
      lambda x: json.loads(re.search(r"```json\s*(\{.*?\})\s*```", x, re.DOTALL).group(1)), # Parse JSON in a ```json``` block
  ]

  for strategy in parsing_strategies:
    try:
      return strategy(text_to_parse)
    except Exception:
            continue


Cáulculo del costo por petición a la API

In [51]:
def computing_cost(response, input_cost, output_cost):
  """
  Input and output cost per million tokens
  """
  num_input_tokens = response.usage_metadata.prompt_token_count
  num_output_tokens = response.usage_metadata.candidates_token_count
  total_cost_per_call=((num_input_tokens)/1000000)*input_cost+((num_output_tokens)/1000000)*output_cost
  print(f'El costo total de la petición fue de: {round(total_cost_per_call,4)} usd \nTokens de entrada:{num_input_tokens}\nTokens de salida:{num_output_tokens}')

Manipulación de la respuesta

In [122]:
from json_repair import repair_json

def response_management(response,output_name):

  print(response.text)

  #Parseo del JSON
  parsed_response = parse_to_dictionary(response.text)

  if print(parsed_response) == None:
    with open('JSON_error.txt',"a") as f:
      f.write(response.text)
      try: # Reparación del JSON
        repaired = repair_json(response.text)
        parsed_response = parse_to_dictionary(repaired)
        print('El archivo JSON se reparó correctamente')
      except: print('El archivo JSON no pudo ser reparado')

  with open(output_name, "w") as f:
    json.dump(parsed_response, f, indent=4, ensure_ascii=False)
    print(f"{output_name} ha sido guardado")

# GenAI

#### Diseño y ejecución del modelo de lenguaje para filtrado de contenido textual, conservando la estructura inicial de la información.

In [120]:
import json
from google import genai
from google.genai import types
from google.colab import userdata

input_file = json_name # Del paso 1

with open(input_file) as json_file:
  json_contents = json_file.read()

client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY_1"))

model_id = "gemini-2.0-flash"

system_prompt = """
Eres un desarrollador experimentado que recibe un archivo JSON con información de un e-commerce con los siguientes atributos:
href, text, title y base_domain. Solo debes tomar la información contenida en "internal".

Tu tarea es generar un JSON limpio removiendo los bloques de información que no hagan referencia a productos.
Remueve los signos de puntuación por la letra sin tilde.
Asegurate de que el JSON sea válido y contenga los corchetes de apertura y cierre, sino completalo.

Incluye contenido relacionado con productos, usualmente se encuentran por categorías. Por ejemplo:
- Salud
- Belleza
- Vehículos
- Alimentos
- Moda

Excluye contenido que tenga información relacionada con:
- Políticas
- Terminos y condiciones
- Redes sociales
- Otras categorías

"""
prompt = f"""
Ten en cuenta que no puedes agregar otras secciones diferentes a las mencionadas en la salida.

Entrada:
"texto: {json_contents}"

la salida debe ser un JSON que mantenga la estructura del archivo de entrada.

Salida:
"href": "url",
"text": "url description",
"title": "título del producto",
"base_domain": "site domain"
"""

response = client.models.generate_content(
    model=model_id,
    contents=prompt,
    config=types.GenerateContentConfig(
        system_instruction=system_prompt,
        temperature=0,
        max_output_tokens=10000,
        candidate_count=1
    )
)

#Cálculo de costos
computing_cost(response,gemini_2_0_flash_input_cost,gemini_2_0_flash_output_cost)

# Nombre del archivo de salida
filtered_json_name = f"{input_file.split('.')[0]}_cleaned.json"

response_management(response,filtered_json_name)


El costo total de la petición fue de: 0.0061 usd 
Tokens de entrada:30276
Tokens de salida:7678
```json
{
  "internal": [
    {
      "href": "https://www.mercadolibre.com.co/categorias#nav-header",
      "text": "Categorias",
      "title": "",
      "base_domain": "mercadolibre.com.co"
    },
    {
      "href": "https://www.mercadolibre.com.co/vehiculos/#menu=categories",
      "text": "Vehiculos",
      "title": "",
      "base_domain": "mercadolibre.com.co"
    },
    {
      "href": "https://www.mercadolibre.com.co/ofertas/supermercado#menu=categories",
      "text": "Supermercado",
      "title": "",
      "base_domain": "mercadolibre.com.co"
    },
    {
      "href": "https://www.mercadolibre.com.co/#/",
      "text": "Tecnologia",
      "title": "",
      "base_domain": "mercadolibre.com.co"
    },
    {
      "href": "https://www.mercadolibre.com.co/c/electrodomesticos#menu=categories",
      "text": "Electrodomesticos",
      "title": "",
      "base_domain": "mercadolibre.

## Paso 3
Se toman las urls de producto identificadas en el paso anterior y extraen markdowns con la información de interés.

Se guardan cada uno de los archivos y también se apilan en la lista *raw_output*

In [124]:
import re
import datetime
import asyncio
from crawl4ai import AsyncWebCrawler

input_file = filtered_json_name # del paso 2

with open(input_file) as json_file:
  json_contents = json_file.read()

parsed_json = parse_to_dictionary(json_contents)

async def main(url):
    # Create an instance of AsyncWebCrawler
    async with AsyncWebCrawler() as crawler:
        # Run the crawler on a URL
        result = await crawler.arun(url)

        # Get the current date and time
        now = datetime.datetime.now()

        # Format the date and time as a string (e.g., "2025-06-23_14-30-00")
        formatted_datetime = now.strftime("%Y-%m-%d_%H-%M-%S")

        # Save the markdown to a file
        if result.success:
            with open(url.split('.')[1] + '_' + formatted_datetime + '.md', "w", encoding="utf-8") as f:
                f.write(result.markdown.raw_markdown)
            print(f"Markdown saved to {url.split('.')[1]}_{formatted_datetime}")
        else:
            print(f"Failed to crawl: {result.error_message}")
    return result

# Run the async main function
raw_output = []

for item in range(len(parsed_json['internal'])):
  output = await(main(url=parsed_json['internal'][item-1]['href'])) #crawl4ai.models.CrawlResultContainer
  raw_output.append(output)

[INIT].... → Crawl4AI 0.6.3 

[FETCH]... ↓ https://listado.mercadolibre.com.co/aire-acondicionado                                               |
✓ | ⏱: 3.76s 

[SCRAPE].. ◆ https://listado.mercadolibre.com.co/aire-acondicionado                                               |
✓ | ⏱: 0.59s 

[COMPLETE] ● https://listado.mercadolibre.com.co/aire-acondicionado                                               |
✓ | ⏱: 4.36s 

Markdown saved to mercadolibre_2025-06-30_20-30-22


...

[INIT].... → Crawl4AI 0.6.3 

[FETCH]... ↓ https://listado.mercadolibre.com.co/air-fryer                                                        |
✓ | ⏱: 3.93s 

[SCRAPE].. ◆ https://listado.mercadolibre.com.co/air-fryer                                                        |
✓ | ⏱: 0.54s 

[COMPLETE] ● https://listado.mercadolibre.com.co/air-fryer                                                        |
✓ | ⏱: 4.49s 

Markdown saved to mercadolibre_2025-06-30_20-37-37


## Paso 4
Se usa un modelo de lenguaje para tomar la información extraida de los markdown de las páginas de producto y extraer la información de interés:



```
"ID": "identificación del producto",
"title": "nombre del producto",
"Precio": "precio del producto",
"imagen_url": "url de la imagen del producto"
"descripción": "descripción del producto"
"base_domain": "site domain"
"base_product_uls": "url del producto"
```



In [134]:
import json
from google import genai
from google.genai import types
from google.colab import userdata

client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY_1"))

model_id = "gemini-2.0-flash"

system_prompt = """
Eres un desarrollador experimentado que recibe un archivo markdown con información de un e-commerce con información de productos.
dentro de las características extraidas se encuentra la sección a la que pertenece y nombre, url de la imagen, url del producto, descripción y precio.

Tu tarea es generar un JSON limpio removiendo los bloques de información que no hagan referencia a productos.
Remueve los signos de puntuación por la letra sin tilde.

Incluye contenido relacionado con productos, usualmente se encuentran por categorías. Por ejemplo:
- Salud
- Belleza
- Vehículos
- Alimentos
- Moda

Excluye contenido que tenga información relacionada con:
- Políticas
- Terminos y condiciones
- Redes sociales

"""
prompt = f"""
Ten en cuenta que no puedes agregar otras secciones diferentes a las mencionadas en la salida.

Entrada:
"texto: {raw_output[2]}"

la salida debe ser un JSON que contenga la información de cada producto encontrado.
No debes agregar información adicional.

Salida:
"ID": "identificación del producto",
"title": "nombre del producto",
"Precio": "precio del producto",
"imagen_url": "url de la imagen del producto"
"descripción": "descripción del producto"
"base_domain": "site domain"
"base_product_uls": "url del producto"
"""

response = client.models.generate_content(
    model=model_id,
    contents=prompt,
    config=types.GenerateContentConfig(
        system_instruction=system_prompt,
        temperature=0,
        max_output_tokens=10000,
        candidate_count=1
    )
)

computing_cost(response,gemini_2_0_flash_input_cost,gemini_2_0_flash_output_cost)

El costo total de la petición fue de: 0.0724 usd 
Tokens de entrada:707910
Tokens de salida:4032


In [135]:
print(response.text)

parsed_response = parse_to_dictionary(response.text)

products_file_json = f"{input_file.split('.')[0]}_products.json"

with open(products_file_json, "w") as f:
  #f.write(response.to_json_dict)
  json.dump(parsed_response, f, indent=4, ensure_ascii=False)
  print(f"{products_file_json} ha sido guardado")

```json
[
  {
    "ID": "MCO2924625584",
    "title": "Renault Captur 2.0 Intens At",
    "Precio": "62000000",
    "imagen_url": "https://http2.mlstatic.com/D_NQ_NP_828420-MCO86951932353_062025-W.jpg",
    "descripcion": "Renault Captur 2.0 Intens At",
    "base_domain": "carro.mercadolibre.com.co",
    "base_product_url": "https://carro.mercadolibre.com.co/MCO-2924625584-renault-captur-20-intens-at-_JM"
  },
  {
    "ID": "MCO2925616776",
    "title": "Bmw X3 3.0 F25 Xdrive35i Executive",
    "Precio": "95800000",
    "imagen_url": "https://http2.mlstatic.com/D_NQ_NP_861209-MCO86681908896_062025-W.jpg",
    "descripcion": "Bmw X3 3.0 F25 Xdrive35i Executive",
    "base_domain": "carro.mercadolibre.com.co",
    "base_product_url": "https://carro.mercadolibre.com.co/MCO-2925616776-bmw-x3-30-f25-xdrive35i-executive-_JM"
  },
  {
    "ID": "MCO1609252095",
    "title": "Toyota Tundra 4.7 Sr5",
    "Precio": "219000000",
    "imagen_url": "https://http2.mlstatic.com/D_NQ_NP_611025-MCO8695